![walmartecomm](walmartecomm.jpg)

Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like the Super Bowl, Labour Day, Thanksgiving, and Christmas. 

In this project, you have been tasked with creating a data pipeline for the analysis of supply and demand around the holidays, along with conducting a preliminary analysis of the data. You will be working with two data sources: grocery sales and complementary data. You have been provided with the `grocery_sales` table in `PostgreSQL` database with the following features:

# `grocery_sales`
- `"index"` - unique ID of the row
- `"Store_ID"` - the store number
- `"Date"` - the week of sales
- `"Weekly_Sales"` - sales for the given store

Also, you have the `extra_data.parquet` file that contains complementary data:

# `extra_data.parquet`
- `"IsHoliday"` - Whether the week contains a public holiday - 1 if yes, 0 if no.
- `"Temperature"` - Temperature on the day of sale
- `"Fuel_Price"` - Cost of fuel in the region
- `"CPI"` – Prevailing consumer price index
- `"Unemployment"` - The prevailing unemployment rate
- `"MarkDown1"`, `"MarkDown2"`, `"MarkDown3"`, `"MarkDown4"` - number of promotional markdowns
- `"Dept"` - Department Number in each store
- `"Size"` - size of the store
- `"Type"` - type of the store (depends on `Size` column)

You will need to merge those files and perform some data manipulations. The transformed DataFrame can then be stored as the `clean_data` variable containing the following columns:
- `"Store_ID"`
- `"Month"`
- `"Dept"`
- `"IsHoliday"`
- `"Weekly_Sales"`
- `"CPI"`
- "`"Unemployment"`"

After merging and cleaning the data, you will have to analyze monthly sales of Walmart and store the results of your analysis as the `agg_data` variable that should look like:

|  Month | Weekly_Sales  | 
|---|---|
| 1.0  |  33174.178494 |
|  2.0 |  34333.326579 |
|  ... | ...  |  

Finally, you should save the `clean_data` and `agg_data` as the csv files.

It is recommended to use `pandas` for this project. 

In [312]:
-- Write your SQL query here
SELECT * FROM grocery_sales

index  Store_ID       Date  Dept  Weekly_Sales
0            0         1 2010-02-05     1      24924.50
1            1         1 2010-02-05    26      11737.12
2            2         1 2010-02-05    17      13223.76
3            3         1 2010-02-05    45         37.44
4            4         1 2010-02-05    28       1085.29
...        ...       ...        ...   ...           ...
231517  232414        24 2011-05-06     8      49471.07
231518  232415        24 2011-05-06    50       1210.00
231519  232416        24 2011-05-06    87      25893.32
231520  232417        24 2011-05-06    85       1357.83
231521  232418        24 2011-05-06    35       3648.91

[231522 rows x 5 columns]

In [313]:
import pandas as pd
import os

# Extract function is already implemented for you 
def extract(store_data, extra_data):
    extra_df = pd.read_parquet(extra_data)
    merged_df = store_data.merge(extra_df, on = "index")
    return merged_df

# Call the extract() function and store it as the "merged_df" variable
merged_df = extract(grocery_sales, "extra_data.parquet")
print(merged_df.columns)

Index(['index', 'Store_ID', 'Date', 'Dept', 'Weekly_Sales', 'IsHoliday',
       'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size'],
      dtype='object')


In [314]:
# Create the transform() function with one parameter: "raw_data"
def transform(raw_data):
    #filling the missing data
    total_missing = raw_data.isna().sum(axis=0)
    misssing_value_columns= list(total_missing[total_missing > 0].index)
    # Write your code here
    for missing in misssing_value_columns:
        raw_data[missing].fillna(raw_data[missing].mode()[0], inplace=True)

    # convert Date series to datime format "%Y-%m-%d"
    # format and extract month in 'Date' field
    date_field = raw_data['Date']
    pd.to_datetime(date_field, format="%Y-%m-%d")
    raw_data['Month'] = date_field.dt.month.astype('int32')

    # keeping the rows where the weekly sales are over $10,000 and dropping the unnecessary columns
    raw_data = raw_data[raw_data['Weekly_Sales'] > 10000]

    # list of the necessary columns
    nec_columns = ['Store_ID', 'Weekly_Sales', 'IsHoliday', 'Month', 'CPI', 'Unemployment']
    # list all column
    columns = list(raw_data.columns)
    # drop unnecessary columns
    for column in columns:
        if column not in nec_columns:
            raw_data.drop(column, axis=1, inplace=True)
    
    return raw_data

In [315]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(merged_df)

# check whether there is any missing data.
assert clean_data.isna().any().any() == False
# check Values of Weekly sale > 10000$
assert (clean_data['Weekly_Sales']>10000).any()
# check added 'Month' field
assert 'Month' in list(clean_data.columns)

In [316]:

# Create the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    mw = clean_data[['Month', 'Weekly_Sales']]
    Avg_Sales = mw.groupby('Month')['Weekly_Sales'].mean()
    agg = Avg_Sales.round(2)
    agg = agg.reset_index()
    agg.rename(columns={"Weekly_Sales": "Avg_Sales"}, inplace=True)
    return agg

In [317]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
agg_data = avg_weekly_sales_per_month(clean_data)
print(agg_data)

    Month  Avg_Sales
0       1   33174.18
1       2   34331.29
2       3   33227.31
3       4   33414.78
4       5   33339.89
5       6   34582.47
6       7   33930.77
7       8   33644.79
8       9   33266.59
9      10   32736.99
10     11   36594.03
11     12   39248.98


In [318]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    full_data.to_csv(full_data_file_path, index=False)
    agg_data.to_csv(agg_data_file_path, index=False)
    pass

In [319]:
# Call the load() function and pass the cleaned and aggregated DataFrames with their paths
load(clean_data, "clean_data.csv", agg_data, "agg_data.csv")

In [320]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(file_path):
    if not os.path.exists(file_path):
        raise Exception(f"File not found: {file_path}")

    print("File exists!")

In [321]:
# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation("clean_data.csv")
validation("agg_data.csv")

File exists!
File exists!
